In [2]:
import os
os.path.join('../')
from collections import defaultdict, OrderedDict
from typing import Dict, List, Any, Callable

from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from doggos.fuzzy_sets import Type1FuzzySet
from doggos.fuzzy_sets.fuzzy_set import FuzzySet
from doggos.induction.information_system import InformationSystem
from doggos.inference import MamdaniInferenceSystem
from doggos.inference.defuzzification_algorithms import center_of_gravity, karnik_mendel
from doggos.inference.inference_system import InferenceSystem
from doggos.knowledge import Rule, Clause, fuzzify, LinguisticVariable, Domain
from doggos.knowledge.consequents import MamdaniConsequent
from doggos.knowledge.consequents.consequent import Consequent
from doggos.utils.grouping_functions import create_set_of_variables
from doggos.utils.membership_functions.membership_functions import generate_equal_gausses, sigmoid, gaussian

In [39]:
dataset_name = "Pima Indians Diabetes"
ds = pd.read_csv("../data/" + dataset_name + ".csv", sep=";")
ds.head()

,F0,F1,F2,F3,F4,F5,F6,F7,Decision
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,43642.0,0.351,31,0
2,8,183,64,0,0,43547.0,0.672,32,1
3,1,89,66,23,94,43493.0,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [40]:
pca = PCA(n_components=4)
values_no_decision = ds.drop(labels=["Decision"], axis=1)
min_max_scaler = StandardScaler()
values_no_decision = min_max_scaler.fit_transform(values_no_decision.values)
pca.fit(values_no_decision)
print(pca.explained_variance_ratio_)

[0.23327578 0.21109452 0.13147107 0.12027503]


In [74]:
pca = KernelPCA(n_components=4, kernel='cosine')
values_no_decision = ds.drop(labels=["Decision"], axis=1)
min_max_scaler = StandardScaler()
values_no_decision = min_max_scaler.fit_transform(values_no_decision.values)
pca.fit(values_no_decision)
print(pca.eigenvalues_ / np.sum(pca.eigenvalues_))

[0.33121241 0.32015514 0.19676206 0.15187038]


In [41]:
principal_components = pca.transform(values_no_decision)
cols = []
for idx in range(principal_components.shape[1]):
    cols.append(f'F{idx}')
pca_pd_ds = pd.DataFrame(principal_components, columns=cols)
pca_pd_ds['Decision'] = ds['Decision']
pca_pd_ds.head()

,F0,F1,F2,F3,Decision
0,-1.563533,-0.675045,-0.446649,0.330954,1
1,1.418342,0.170636,0.040022,0.429542,0
2,-0.152949,-1.766084,1.227255,-1.360971,1
3,1.555166,0.763236,0.288356,-0.016735,0
4,0.017222,1.580277,0.070721,-0.750512,1


In [42]:
pca_pd_ds.to_csv("../data/" + dataset_name + " StdPCA.csv", sep=";", index=False)
